In [1]:
import sys 
sys.path.append('/workspace/Documents')
import os
import torch
import numpy as np
import nibabel as nb
import CTDenoising_Diffusion_N2N.noise2noise.model as noise2noise
import CTDenoising_Diffusion_N2N.functions_collection as ff
import CTDenoising_Diffusion_N2N.Build_lists.Build_list as Build_list
import CTDenoising_Diffusion_N2N.noise2noise.Generator as Generator

/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Step 1: define pre-trained model

In [2]:
trial_name = 'noise2noise'
epoch = 78
trained_model_filename = os.path.join('/mnt/camca_NAS/denoising/models', trial_name, 'models/model-' + str(epoch)+ '.pt')
save_folder = os.path.join('/mnt/camca_NAS/denoising/models', trial_name, 'pred_images'); os.makedirs(save_folder, exist_ok=True)

In [3]:
### parameters no need to change
image_size = [512,512]

histogram_equalization = True
background_cutoff = -1000
maximum_cutoff = 2000
normalize_factor = 'equation' 

### step 2: build patient list - testing, batch 5

In [4]:
build_sheet =  Build_list.Build(os.path.join('/mnt/camca_NAS/denoising/Patient_lists/fixedCT_static_simulation_train_test_gaussian_NAS.xlsx'))
_,patient_id_list,patient_subid_list,random_num_list, condition_list, x0_list = build_sheet.__build__(batch_list = [5]) 

### step 3: build model

In [5]:
# build model
model = noise2noise.Unet2D(
    init_dim = 16,
    channels = 2, 
    out_dim = 1,
    dim_mults = (2,4,8,16),
    full_attn = (None,None, False, True),
    act = 'ReLU',
)

in out is :  [(16, 32), (32, 64), (64, 128), (128, 256)]


### step 4: main

In [6]:
# load histogram equalization pre-saved files
bins = np.load('/mnt/camca_NAS/denoising/Data/histogram_equalization/bins.npy') # change to your own path, they are in this repo as well (example data)
bins_mapped = np.load('/mnt/camca_NAS/denoising/Data/histogram_equalization/bins_mapped.npy')

In [7]:
# main
for i in range(0, 1):#patient_id_list.shape[0]):
    patient_id = patient_id_list[i]
    patient_subid = patient_subid_list[i]
    random_num = random_num_list[i]
    x0_file = x0_list[i]
    condition_file = condition_list[i]

    print(i,patient_id, patient_subid, random_num)

    # # get the ground truth image
    # gt_img = nb.load(x0_file)
    # affine = gt_img.affine; gt_img = gt_img.get_fdata()[:,:,30:80]

    # get the condition image
    condition_img = nb.load(condition_file).get_fdata()[:,:,30:80]
    affine = nb.load(condition_file).affine
    nb.save(nb.Nifti1Image(condition_img, affine), os.path.join(save_folder, 'condition_img.nii.gz'))

    # make folders
    ff.make_folder([os.path.join(save_folder, patient_id), os.path.join(save_folder, patient_id, patient_subid), os.path.join(save_folder, patient_id, patient_subid, 'random_' + str(random_num))])
    save_folder_case = os.path.join(save_folder, patient_id, patient_subid, 'random_' + str(random_num), 'epoch' + str(epoch)); os.makedirs(save_folder_case, exist_ok=True)

    # # generator
    generator = Generator.Dataset_2D(
        img_list = np.array([condition_file]),
        image_size = image_size,

        num_slices_per_image = 50,
        random_pick_slice = False,
        slice_range = [30,80],

        bins = bins,
        bins_mapped = bins_mapped,
        histogram_equalization = histogram_equalization,
        background_cutoff = background_cutoff,
        maximum_cutoff = maximum_cutoff,
        normalize_factor = normalize_factor,)

    # # sample:
    sampler = noise2noise.Sampler(model,generator,batch_size = 1, image_size = image_size)

    pred_img = sampler.sample_2D(trained_model_filename, condition_img)
    pred_img_final = pred_img
    
    # save
    nb.save(nb.Nifti1Image(pred_img_final, affine), os.path.join(save_folder_case, 'pred_img.nii.gz'))

0 00214841 0000455418 0


histogram equalization:  True


/workspace/Documents/CTDenoising_Diffusion_N2N/noise2noise/model.py:763: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  data = torch.load(trained_model_filename, map_location

model device:  cuda:0
gt_img shape:  (512, 512, 50)
load image file:  /mnt/camca_NAS/denoising/Data/simulation/00214841/0000455418/gaussian_random_0/recon.nii.gz
